In [2]:
import tensorflow as tf
import os
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    raise RuntimeError("GPU tidak ditemukan.")

In [4]:
IMG_SIZE = 299
BATCH_SIZE = 32

In [5]:
image_paths = glob.glob('Data/food-101/images/*/*.jpg')
labels = [os.path.split(os.path.split(path)[0])[1] for path in image_paths]
unique_labels = sorted(set(labels))
label_to_index = {label: index for index, label in enumerate(unique_labels)}
labels = [label_to_index[label] for label in labels]

def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
train_data = tf.data.Dataset.from_tensor_slices((train_paths, train_labels)).map(load_and_preprocess).batch(BATCH_SIZE)
test_data = tf.data.Dataset.from_tensor_slices((test_paths, test_labels)).map(load_and_preprocess).batch(BATCH_SIZE)

In [6]:
input = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = EfficientNetB1(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model(input)

x = GlobalAveragePooling2D()(x)
x = Dropout(0.05)(x)

predictions = Dense(101, activation='softmax')(x)

model = Model(inputs=input, outputs=predictions)

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 efficientnetb1 (Functional)  (None, 10, 10, 1280)     6575239   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
Total params: 6,704,620
Trainable params: 129,381
Non-trainable params: 6,575,239
_____________________________________________

In [8]:
history = model.fit(train_data, epochs=4, validation_data=test_data)

Epoch 1/4
2525/2525 [==============================] - 734s 275ms/step - loss: 1.6389 - accuracy: 0.5969 - val_loss: 1.2264 - val_accuracy: 0.6834
Epoch 2/4
2525/2525 [==============================] - 689s 273ms/step - loss: 1.1579 - accuracy: 0.6953 - val_loss: 1.1322 - val_accuracy: 0.7048
Epoch 3/4
2525/2525 [==============================] - 690s 273ms/step - loss: 1.0391 - accuracy: 0.7226 - val_loss: 1.0940 - val_accuracy: 0.7147
Epoch 4/4
2525/2525 [==============================] - 689s 273ms/step - loss: 0.9652 - accuracy: 0.7397 - val_loss: 1.0783 - val_accuracy: 0.7188


In [9]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        if logs.get('val_accuracy') > 0.80:
            print("\nAkurasi telah mencapai 80%, menghentikan pelatihan.")
            self.model.stop_training = True

callback = MyCallback()

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 efficientnetb1 (Functional)  (None, 10, 10, 1280)     6575239   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
Total params: 6,704,620
Trainable params: 2,474,421
Non-trainable params: 4,230,199
___________________________________________

In [12]:
history = model.fit(train_data, epochs=1, validation_data=test_data, callbacks=callback)

2525/2525 [==============================] - 756s 299ms/step - loss: 0.4493 - accuracy: 0.8714 - val_loss: 1.0418 - val_accuracy: 0.7508


In [14]:
model.save_weights("model.h5")